In [1]:
import json

#### Read Dataset & Entities

In [2]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [3]:
test_source_path = '/home/ml/users/cadencao/XSum/fairseq_files/train.source'
test_target_path = '/home/ml/users/cadencao/XSum/fairseq_files/train.target'

xsum_source = read_lines(test_source_path)
xsum_target = read_lines(test_target_path)

In [4]:
xsum_ents = json.load(open('xsum_train_ents.json', 'r'))

In [5]:
assert len(xsum_source) == len(xsum_target) == len(xsum_ents)
print(len(xsum_target))

203575


#### Build Clean Dataset

In [6]:
from tqdm import tqdm

In [7]:
train_src_clean, train_tgt_clean = [], []
labels = []

for i in tqdm(range(len(xsum_source))):
    tgt_ents = [xsum_target[i][e['start']: e['end']].lower() for e in xsum_ents[i]['ents']]
    
    entiled = True
    for te in tgt_ents:
        if te not in xsum_source[i].lower():
            entiled = False
            labels.append(0)
            break
    
    if entiled:
        labels.append(1)
        train_src_clean.append(xsum_source[i])
        train_tgt_clean.append(xsum_target[i])

100%|██████████| 203575/203575 [00:02<00:00, 97273.06it/s]


In [8]:
print(len(train_src_clean))
assert len(train_src_clean) == len(train_tgt_clean)

54019


#### Save Data

In [9]:
import numpy as np

In [10]:
labels = np.array(labels)
print(labels.shape)

(203575,)


In [11]:
directory = '/home/ml/users/cadencao/XSum/clean_files/'

np.save(directory + 'clean_data_labels.npy', labels)    # .npy extension is added if not given

with open(directory + 'train.source', 'w', encoding='utf-8') as sf:
    with open(directory + 'train.target', 'w', encoding='utf-8') as tf:
        for s, t in zip(train_src_clean, train_tgt_clean):
            sf.write(s + '\n')
            tf.write(t + '\n')